### 베이스라인 모델은 다음과 같은 이유로 선정하였습니다.
- 데이터 특성 상, 피처가 많아 다중공선성의 위험이 있고, 범주형 변수가 많았습니다.
- 따라서, 다중공선성에 강하고 규제화가 잘 되며 범주형 변수에 강한 모델을 선정하였습니다.
  
- **ARDRegression**
특징: 각 특성(feature)에 대해 별도의 정규화 강도를 추정해 불필요한 변수를 자동으로 축소(/제거)  
장점: 다중공선성(collinearity)에 강하며, 모델 간결성 유지  

- **XGBRegressor**
특징: 트리 기반 GBM의 최적화판, 병렬 학습·정규화·결측 처리 내장  
장점: 속도·성능 우수, 과적합 제어용 파라미터 풍부, n_jobs 지원  

- **LGBMRegressor**
특징: histogram 기반, 잎 중심(leaf-wise) 분할 → 대용량 데이터에 빠름  
장점: 메모리 효율·학습 속도 우수, n_jobs 지원  

- **CatBoostRegressor**
특징: 범주형 변수를 별도 전처리 없이 자동 처리, ordered boosting으로 과적합 억제  
장점: 범주형 데이터 다룰 때 성능·편의성 우수, 튜닝 부담 비교적 낮음  

In [1]:
import pandas as pd
df = pd.read_parquet(
    'data.parquet',
    engine='pyarrow'         # 저장 시 사용한 엔진과 동일하게 지정
)
test_loaded = pd.read_parquet(
    'test.parquet',
    engine='pyarrow'         # 저장 시 사용한 엔진과 동일하게 지정
)

# 대회 공지사항에 따라 df 이상치 드롭
- 남위례를 제외한 한대앞~오이도역 구간은 내부 프로그램 오류로 인하여 22년 6월 13일까지 4호선 재차인원이 누락되었습니다. 
- 해당기간동안 한대앞~오이도역을 이용하는 인원은 모두 수인분당선을 이용하는것으로 기록되었습니다.
- 남위례역은 21년 12월 18일에 개통하였으며, 프로그램 내부에 개통사항 반영이 늦어져 혼잡도가 0으로 산출된 것으로 확인됩니다.

In [2]:
# feature / target 정의
ordered_cols = ['Direction', 'time_period']
cat_cols     = [
                'station_number'
                , 'address'
               ] + ordered_cols
num_cols = [
    'HM','RN_DAY','RN_HR1',
    'TA','WD','WS'
    ,'STN'
    ,'sin_dom','cos_dom','sin_dow','cos_dow','sin_hod','cos_hod'
    ,'sin_wom','cos_wom','sin_woy','cos_woy','sin_doy','cos_doy'
    ,'day','day_of_year','hour'
    ,'is_day_before_holiday','is_day_after_holiday','is_holiday','is_weekend'
    ,'month','transfer','week_of_month','week_of_year','weekday','year'
    ,'신설역', '신규관측소'
]
feature_cols = num_cols + ordered_cols + cat_cols
target_col   = 'Congestion'

# 모델 선택

In [3]:
def evaluate_model(name, model, X_train, y_train, X_val, y_val):
    t0 = time.time()
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_val)
    elapsed = time.time() - t0
    
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    r2   = r2_score(y_val, y_pred)
    
    return {'Model': name, 'Time(s)': elapsed, 'RMSE': rmse, 'R2': r2}

In [4]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm

# 전처리·평가용
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics      import mean_squared_error, r2_score

# ── 선형 계열 회귀 모델 ──
from sklearn.linear_model import ARDRegression

# ── 트리 & 앙상블 ──
from sklearn.ensemble      import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    GradientBoostingRegressor
)

# ── 신경망 & 부스팅 ──
from sklearn.neural_network import MLPRegressor
from xgboost                 import XGBRegressor
from lightgbm                import LGBMRegressor
from catboost                import CatBoostRegressor

# ------------------------------------------------------------------------------
# 미리 정의해야 할 변수
# df: 학습용 DataFrame (컬럼에 'Line', 'TM', STN, address, feature_cols, target_col 포함)
# test: 테스트용 DataFrame (컬럼 구조 동일)
# feature_cols: predictor로 사용할 컬럼 리스트
# target_col: 예측 대상 컬럼 이름 (문자열)
# cat_cols: 범주형으로 one-hot encoding 할 컬럼 리스트 (예: ['STN','address'])
# ------------------------------------------------------------------------------

def evaluate_model(name, model, X_train, y_train, X_val, y_val):
    t0 = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return {
        'Model': name,
        'Time(s)': time.time() - t0,
        'RMSE': np.sqrt(mean_squared_error(y_val, y_pred)),
        'R2': r2_score(y_val, y_pred)
    }

all_results = []

for line in range(1, 9):
    # 1) subset & sort
    df_line   = df [df['Line']==line].sort_values('TM').copy()
    test_line = test_loaded[test_loaded['Line']==line].copy()

    # 2) 카테고리 지정
    for col in cat_cols:
        df_line[col]   = df_line[col].astype('category')
        test_line[col] = test_line[col].astype('category')

    # 3) feature & target
    X      = df_line[feature_cols]
    y      = df_line[target_col].astype(int)
    X_test = test_line[feature_cols]

    # 4) 원-핫 인코딩
    X_enc      = pd.get_dummies(X,      columns=cat_cols, drop_first=False)
    X_test_enc = pd.get_dummies(X_test, columns=cat_cols, drop_first=False)

    # 5) 중복 컬럼 제거 & 정렬, 누락 채움
    X_enc      = X_enc.loc[:, ~X_enc.columns.duplicated()]
    X_test_enc = X_test_enc.loc[:, ~X_test_enc.columns.duplicated()]
    X_test_enc = X_test_enc.reindex(columns=X_enc.columns, fill_value=0)

    # 6) 정규화
    mm             = MinMaxScaler()
    X_scaled       = mm.fit_transform(X_enc)
    X_test_scaled  = mm.transform(X_test_enc)

    # 7) 시간 순 분할 (train:val = 8:2)
    split_idx = int(len(X_scaled) * 0.8)
    X_train, X_val = X_scaled[:split_idx], X_scaled[split_idx:]
    y_train, y_val = y.values[:split_idx],    y.values[split_idx:]

    # 8) 모델별 평가
    for name, model in [
        ('ARD',  ARDRegression()),
        ('XGB',  XGBRegressor(tree_method='hist', n_jobs=-1, random_state=42)),
        # ('LGBM', LGBMRegressor(n_jobs=-1, random_state=42)),
        # ('CAT',  CatBoostRegressor(verbose=0, random_state=42))
    ]:
        res = evaluate_model(name, model, X_train, y_train, X_val, y_val)
        res['Line'] = line
        all_results.append(res)
        print(f"[Line {line}] {name}: RMSE={res['RMSE']:.3f}, R2={res['R2']:.3f}, Time={res['Time(s)']:.1f}s")

# 9) 종합 결과 DataFrame 생성 및 저장
results_df = pd.DataFrame(all_results)
print("\n=== 전체 라인·모델별 실행 시간·성능 비교 ===")
print(results_df)

# CSV로 저장 (필요시)
os.makedirs('results', exist_ok=True)
results_df.to_csv('results/model_performance_all_lines.csv', index=False)

[Line 1] ARD: RMSE=15.080, R2=0.459, Time=180.7s
[Line 1] XGB: RMSE=6.857, R2=0.888, Time=23.6s
[Line 2] ARD: RMSE=15.616, R2=0.468, Time=146.2s
[Line 2] XGB: RMSE=7.881, R2=0.864, Time=15.0s
[Line 3] ARD: RMSE=13.590, R2=0.531, Time=123.6s
[Line 3] XGB: RMSE=5.669, R2=0.918, Time=13.6s
[Line 4] ARD: RMSE=15.093, R2=0.489, Time=321.0s
[Line 4] XGB: RMSE=6.823, R2=0.896, Time=15.9s
[Line 5] ARD: RMSE=13.581, R2=0.511, Time=105.1s
[Line 5] XGB: RMSE=6.083, R2=0.902, Time=16.2s
[Line 6] ARD: RMSE=11.456, R2=0.470, Time=52.3s
[Line 6] XGB: RMSE=5.284, R2=0.887, Time=12.8s
[Line 7] ARD: RMSE=17.961, R2=0.383, Time=149.1s
[Line 7] XGB: RMSE=8.444, R2=0.864, Time=19.7s
[Line 8] ARD: RMSE=15.102, R2=0.590, Time=25.6s
[Line 8] XGB: RMSE=5.884, R2=0.938, Time=4.6s

=== 전체 라인·모델별 실행 시간·성능 비교 ===
   Model     Time(s)       RMSE        R2  Line
0    ARD  180.651611  15.080387  0.459299     1
1    XGB   23.625844   6.857039  0.888209     1
2    ARD  146.226538  15.615536  0.467739     2
3    XGB   1